In [2]:
import ee
import pandas as pd

In [3]:
ee.Authenticate()

Enter verification code:  4/1AeaYSHDoVBY2Q8CcVzmDyqZ0QELcw_zDPs83Rn-rZx0mzpwfiwIWBh7b8Hc



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [13]:
canopy_height = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1')
canopy_height.bandNames().getInfo()

['b1']

In [19]:
sd_canopy = ee.Image('users/nlang/ETH_GlobalCanopyHeightSD_2020_10m_v1')
sd_canopy.bandNames().getInfo()

['b1']

In [9]:
#This is the location of the flux sites, later this should be read from the folder.
target_location = (2.284102, 47.322918) 

In [22]:
# Define the target location
target_location = ee.Geometry.Point(2.284102, 47.322918) #ee.Geometry.Point(lng, lat)

# Sample the canopy height image at the target location
height_canopy = canopy_height.sample(target_location, 10).first().get('b1').getInfo()  # Change 'b1' to appropriate band name if needed
sd_height_canopy = sd_canopy.sample(target_location, 10).first().get('b1').getInfo()
# Print the value
print('Canopy height value at target location:', height_canopy)
print('sd of Canopy height value at target location:', sd_height_canopy)

Canopy height value at target location: 15
sd of Canopy height value at target location: 7


In [18]:
sample_value

15